<a href="https://colab.research.google.com/github/aggarwal-ujjwal/AI-ML/blob/main/Deep_Q_Learning_for_Lunar_Landing_Handwritten_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://gymnasium.farama.org/environments/box2d/lunar_lander/

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 3s (414 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [ ]:
import os
import random
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from collections import deque, namedtuple



Creating the NN

In [7]:
class NeuralNetwork(nn.Module): #inheritance
  def __init__(self, input_size, nb_action, seed = 42): #you can remove seed from here
    super(NeuralNetwork, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(input_size, 64) #64 has to be calculated with expermentation
    self.fc2 = nn.Linear(64,64)
    self.fc3 = nn.Linear(64, nb_action) #ends with 4 possible actions
  #noob way
  # def forward(self, state):
  #   x = self.fc1(state)
  #   x = F.relu(x)
  #   x = self.fc2(x)
  #   x = F.relu(x)
  #   return self.fc3(x)

  #better way
  def forward(self, input): #input vector with 8 values like- ([0.5, -0.2, 1.0, 0.8])
    """ # fc1(state): [8] -> [64] (linear transformation + bias)
        # F.relu(): applies ReLU activation (max(0, x))
        # x shape: [64] with all negative values clipped to 0"""
    x = F.relu(self.fc1(input))
    x = F.relu(self.fc2(x))
    actions = self.fc3(x)
    return actions


In [12]:
#Example for understanding - not to be used anywhere
x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])
fc = nn.Linear(10, 64)
output = fc(x)
print(output)
print(x.shape, output.shape)

tensor([-7.3435,  0.9240, -0.5183, -3.0734, -4.8965,  4.1848,  2.1088,  3.0115,
        -1.8391,  2.1751, -1.0464, -0.9028, -0.4862,  0.5820, -2.7835,  4.8207,
         0.7896, -4.4496, -1.0755,  1.4353, -2.6281,  2.2394,  0.7500, -5.1116,
        -0.5933, 10.4399, -3.7471, -0.2170, -4.4096, -2.6567,  4.2698, -4.7741,
         3.1276,  4.7805, -2.8647,  4.6988,  1.5122,  1.6034,  1.8067, -2.9319,
         4.5249,  3.7636, -0.9490, -0.9455,  1.1468,  1.4946, -2.7114, -3.1452,
        -1.5391, -9.4637,  1.4935, -2.7736,  2.1752,  0.3353, -3.9348, -2.6553,
         7.2100, -2.7753, -7.0117, -5.7095,  1.3408, -1.7578, -7.9469, -7.0600],
       grad_fn=<ViewBackward0>)
torch.Size([10]) torch.Size([64])
